In [1]:
import datetime
import numpy as np
from shapely.geometry.point import Point
from skimage.draw import circle_perimeter_aa
from matplotlib import pyplot as plt
%matplotlib inline
from pathlib import Path
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.metrics import MeanSquaredError # why two options?
from tensorflow.keras.utils import get_file
import pandas as pd
from PIL import Image

In [2]:
tf.__version__

'2.1.0'

In [10]:
column_names = ['image_path', 'row', 'col', 'rad']
label_names = column_names[1:] # there are three predictions... the feature is actually the image
feature_name = ['image_path']

In [11]:
label_names

['row', 'col', 'rad']

In [12]:
img_height = 200
img_width = 200
batch_size = 16

In [13]:
train_label_df = pd.read_csv(labels_path)

In [14]:
train_label_df.columns

Index(['col', 'image_path', 'rad', 'row'], dtype='object')

In [15]:
train_label_df

,col,image_path,rad,row
0,179.0,0.png,38.0,102.0
1,2.0,1.png,22.0,51.0
2,67.0,2.png,12.0,15.0
3,198.0,3.png,27.0,146.0
4,46.0,4.png,40.0,94.0
...,...,...,...,...
995,63.0,995.png,24.0,118.0
996,196.0,996.png,11.0,104.0
997,195.0,997.png,36.0,174.0
998,17.0,998.png,25.0,1.0


In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255) 

In [17]:
train_generator = train_datagen.flow_from_dataframe(dataframe=train_label_df, directory=images_path, 
                                              x_col="image_path", y_col=label_names, class_mode="other",
                                                    target_size=(img_width, img_height), 
                                              batch_size=batch_size)

Found 100 validated image filenames.


C:\Users\Julius\anaconda3\envs\tf\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 900 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [18]:
# should i make a test generator?
#image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [19]:
image_count = len(list(images_path.glob('*.png')))
image_count

100

In [30]:
model = Sequential()
model.add(Conv2D(32, 3, strides=(1,1), activation='relu', input_shape=(200, 200, 2)))
model.add(Conv2D(32, 3, strides=(1,1), activation='relu'))
model.add(MaxPool2D(pool_size=(4,4)))
model.add(Conv2D(64, 3, strides=(1,1), activation='relu'))
model.add(Conv2D(64, 3, strides=(1,1), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(128, 3, strides=(1,1), activation='relu'))
model.add(Conv2D(128, 3, strides=(1,1), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 198, 198, 32)      608       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 196, 196, 32)      9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 45, 45, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 20, 20, 128)      

In [32]:
experiment_name = ''

In [33]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_path = Path(r'C:\Users\Julius\tensorboard\scale') / experiment_name / timestamp
#tensorboard_path += '123'

In [34]:
tensorboard_path

WindowsPath('C:/Users/Julius/tensorboard/scale/20200705-184634')

In [35]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_path, write_images=True, update_freq='batch')

In [40]:
a = next(iter(train_generator))

In [42]:
a

(array([[[[1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          [0.2392157 , 0.2392157 , 0.2392157 ]],
 
         [[0.5529412 , 0.5529412 , 0.5529412 ],
          [1.        , 1.        , 1.        ],
          [1.        , 1.        , 1.        ],
          ...,
          [1.        , 1.        , 1.        ],
          [0.28627452, 0.28627452, 0.28627452],
          [0.16078432, 0.16078432, 0.16078432]],
 
         [[0.9803922 , 0.9803922 , 0.9803922 ],
          [0.96470594, 0.96470594, 0.96470594],
          [1.        , 1.        , 1.        ],
          ...,
          [0.5019608 , 0.5019608 , 0.5019608 ],
          [0.64705884, 0.64705884, 0.64705884],
          [0.7568628 , 0.7568628 , 0.7568628 ]],
 
         ...,
 
         [[1.        , 1.        , 1.        ],
          [1.     

In [43]:
len(a)

2

In [44]:
a[0]

array([[[[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         [0.2392157 , 0.2392157 , 0.2392157 ]],

        [[0.5529412 , 0.5529412 , 0.5529412 ],
         [1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ],
         ...,
         [1.        , 1.        , 1.        ],
         [0.28627452, 0.28627452, 0.28627452],
         [0.16078432, 0.16078432, 0.16078432]],

        [[0.9803922 , 0.9803922 , 0.9803922 ],
         [0.96470594, 0.96470594, 0.96470594],
         [1.        , 1.        , 1.        ],
         ...,
         [0.5019608 , 0.5019608 , 0.5019608 ],
         [0.64705884, 0.64705884, 0.64705884],
         [0.7568628 , 0.7568628 , 0.7568628 ]],

        ...,

        [[1.        , 1.        , 1.        ],
         [1.        , 1.        , 1.        ]

In [45]:
a[0].shape

(16, 200, 200, 3)

In [37]:
model.compile(optimizer = Adam(),
             loss = MeanSquaredError(),
             metrics = ['mae', 'mse'])

In [38]:
# should only be 1 epoch because better to just make more training data
num_epochs = 5

Should only need one epoch

In [39]:
history = model.fit(x=train_generator, epochs=num_epochs, callbacks=[tensorboard_callback])

  ...
    to  
  ['...']
Train for 7 steps
Epoch 1/5
1/7 [===>..........................] - ETA: 4s

InvalidArgumentError:  input depth must be evenly divisible by filter depth: 3 vs 2
	 [[node sequential_1/conv2d_6/Conv2D (defined at <ipython-input-39-f4177c1d6f52>:1) ]] [Op:__inference_distributed_function_2712]

Function call stack:
distributed_function


In [ ]:
test_im = r'E:\Data\Raw\Circles\images\0.png'

In [ ]:
def preprocess_image(im):
    im = Image.open(im)
    im_arr = np.array(im)
    assert im_arr.shape == (200, 200, 4)
    # This gives us a fourth channel of 255. We'll want to remove it to match the shape.
    im_arr = im_arr[:, :, :3]
    # Make it a batch of one image
    im_arr.shape = (1, 200, 200, 3)
    # normalize it
    im_arr = im_arr / 255.
    return im_arr

In [ ]:
im_arr = preprocess_image(test_im)

In [ ]:
model.predict(im_arr)

Here's the evaluation

In [ ]:
noisy_circle(200, 50, 2)

In [ ]:
params, img = noisy_circle(200, 50, 2)

In [ ]:
params # height, width, radius

In [ ]:
img.shape

In [ ]:
a = Image.fromarray(img, 'L')

In [ ]:
a

In [ ]:
np.array(a).shape

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow??